<a href="https://colab.research.google.com/github/Revivo25/Computing-Cloud/blob/main/EX5Cloud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install firebase


In [ ]:
# ===== Install libraries (in Colab) =====
# Run this once in a separate cell:
# !pip install firebase firebase-admin

# ===== Import library and connect to Firebase =====
from firebase import firebase

# Your Firebase Realtime Database URL:
FBconn = firebase.FirebaseApplication(
    'https://test-42d3e-default-rtdb.firebaseio.com/', None
)

# Base path in the DB for the words
WORDS_PATH = '/words'


# ===== Helper functions for the DB =====

def get_all_words():
    """Return all words from the DB as a dict."""
    records = FBconn.get(WORDS_PATH, None)
    if records is None:
        return {}
    return records


def add_single_word(word):
    """Add a single word: if it exists, increase count; otherwise create it."""
    word = word.strip().lower()
    if not word:
        return

    records = get_all_words()
    current_count = 0

    if word in records and 'count' in records[word]:
        current_count = records[word]['count']

    new_count = current_count + 1
    # put creates/updates a node named "word" under /words
    FBconn.put(WORDS_PATH, word, {'count': new_count})
    print(f"'{word}' updated to count = {new_count}")


def add_text(text):
    """Add a full text: count each word and update the DB."""
    words = text.split()
    for w in words:
        add_single_word(w)


def update_word_count(word, new_count):
    """Manually update the count for a given word."""
    word = word.strip().lower()
    if not word:
        print("Empty word. Nothing to update.")
        return

    FBconn.put(WORDS_PATH, word, {'count': new_count})
    print(f"'{word}' count was set to {new_count}")


def delete_word(word):
    """Delete a word from the DB."""
    word = word.strip().lower()
    if not word:
        print("Empty word. Nothing to delete.")
        return

    FBconn.delete(WORDS_PATH, word)
    print(f"'{word}' deleted from DB")


def show_all_words():
    """Print all words and their counts."""
    records = get_all_words()
    if not records:
        print("No words stored in the database.")
        return

    print("\n=== All words in the database ===")
    for word, data in records.items():
        count = data.get('count', 0)
        print(f"{word}: {count}")
    print("=================================")


# ===== Main menu =====

def main_menu():
    while True:
        print("\n----- Word Counter Menu -----")
        print("1. Add single word")
        print("2. Add full text")
        print("3. Update word count")
        print("4. Delete word")
        print("5. Show all words")
        print("6. Exit")

        choice = input("Choose an option (1-6): ")

        if choice == '1':
            word = input("Enter a word: ")
            add_single_word(word)

        elif choice == '2':
            text = input("Enter a full text: ")
            add_text(text)

        elif choice == '3':
            word = input("Which word do you want to update? ")
            try:
                new_count = int(input("Enter the new count: "))
                update_word_count(word, new_count)
            except ValueError:
                print("You must enter an integer number.")

        elif choice == '4':
            word = input("Which word do you want to delete? ")
            delete_word(word)

        elif choice == '5':
            show_all_words()

        elif choice == '6':
            print("Goodbye")
            break

        else:
            print("Invalid choice. Please try again.")


# Run the menu:
main_menu()
